In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from  tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import shutil
import time
#import torch
#import cv2 as cv2
#from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML

In [3]:
c=0
for filename in os.listdir("../archive/images"):
    if filename.endswith(".jpg"):
        c+=1    
print(c)


10015


In [4]:
groundTruth = pd.read_csv('../archive/GroundTruth.csv')

In [5]:
groundTruth

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Check to see if the categories are mutually exclusive
sumRows = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=1) 
#Add up each row. If mutually exclusive sum of each row should be 1

print(sumRows.sum(axis=0))
print(len(sumRows))

10015.0
10015


In [7]:
sumColumns = pd.DataFrame(groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
sumColumns = sumColumns.rename(columns={0: "Total"})
sumColumns['Percent Total'] = sumColumns['Total'] / len(groundTruth)

In [8]:
sumColumns
#Very skewed data. Will need to do stratified

,Total,Percent Total
MEL,1113.0,0.111133
NV,6705.0,0.669496
BCC,514.0,0.051323
AKIEC,327.0,0.032651
BKL,1099.0,0.109735
DF,115.0,0.011483
VASC,142.0,0.014179


In [81]:
#trainingDf, testingDf= train_test_split(groundTruth, test_size=0.30, random_state=42, stratify = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]])
#Stratify on each outcome

testingDf, trainingDf  = train_test_split( groundTruth, test_size=0.013, random_state=42, stratify = groundTruth[["NV"]])
#Stratify on each outcome

#Training DF will be 40% of the data set
#Testing data set will be split again to make a validation and testing data set, (30% each)





In [82]:
trainingDf

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
3165,ISIC_0027471,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2398,ISIC_0026704,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7154,ISIC_0031460,0.0,1.0,0.0,0.0,0.0,0.0,0.0
854,ISIC_0025160,0.0,1.0,0.0,0.0,0.0,0.0,0.0
395,ISIC_0024701,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2659,ISIC_0026965,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9352,ISIC_0033658,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6470,ISIC_0030776,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4611,ISIC_0028917,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# Double check to see if stratification worked on the train set
trainSumColumns = pd.DataFrame(trainingDf[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
trainSumColumns = trainSumColumns.rename(columns={0: "Total"})
trainSumColumns['Percent Total'] = trainSumColumns['Total'] / len(trainingDf)
trainSumColumns

,Total,Percent Total
MEL,15.0,0.114504
NV,88.0,0.671756
BCC,1.0,0.007634
AKIEC,4.0,0.030534
BKL,18.0,0.137405
DF,4.0,0.030534
VASC,1.0,0.007634


In [84]:
# Double check to see if stratification worked on the test set
testSumColumns = pd.DataFrame(testingDf[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
testSumColumns = testSumColumns.rename(columns={0: "Total"})
testSumColumns['Percent Total'] = testSumColumns['Total'] / len(testingDf)
testSumColumns

,Total,Percent Total
MEL,1098.0,0.111089
NV,6617.0,0.669466
BCC,513.0,0.051902
AKIEC,323.0,0.032679
BKL,1081.0,0.109369
DF,111.0,0.011230
VASC,141.0,0.014265


In [65]:
# trainingDf = trainingDf[["image", "NV"]]
# testingDf = testingDf[["image", "NV"]]

In [66]:
#Just a simple test data set for setting up the models
train = trainingDf.iloc[1:15,:]

test = testingDf.iloc[1:10,:]

In [85]:
# Make a list to store all of the processed images
x_data_train = []
# y_d_test = []


#Process all the images in the training data set.
for i in trainingDf.iloc[:,0]: #FIXME CHANGE BACK TO TRAINING DF
    path="../archive/images/" + i + ".jpg"
    
    image=tf.keras.preprocessing.image.load_img(
        path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
    input_arr = tf.keras.preprocessing.image.img_to_array(image)

    x_data_train.append(input_arr)
    
#     print(trainingDf.loc[trainingDf['image'] == i]["NV"])
#     y_d_test.append(trainingDf.loc[trainingDf['image'] == i]["NV"])




In [ ]:
x_data_test = []

for i in test.iloc[:,0]: #FIXME CHANGE BACK TO TESTING DF
    path="../archive/images/" + i + ".jpg"
    
    image=tf.keras.preprocessing.image.load_img(
        path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
    input_arr = tf.keras.preprocessing.image.img_to_array(image)

    x_data_test.append(input_arr)

In [86]:
x_data_train[0].shape


(450, 600, 3)

In [87]:
#Start Building model

shape = x_data_train[0].shape

In [93]:
# Simple dummy model
def buildModel():
    model = keras.models.Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape=shape))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(rate=0.5))
    model.add(Dense(7, activation="softmax"))
    return model
    


In [94]:
m = buildModel()
m.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 446, 596, 32)      2432      
                                                                 
 conv2d_12 (Conv2D)          (None, 442, 592, 32)      25632     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 221, 296, 32)     0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 221, 296, 32)      0         
                                                                 
 flatten_4 (Flatten)         (None, 2093312)           0         
                                                                 
 dense_8 (Dense)             (None, 256)               535888128 
                                                      

In [95]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01)
m.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])


In [96]:


#Get y-Data ready for model
y_train_final = np.asarray(trainingDf.loc[:, trainingDf.columns != 'image'])
# y_train_final = np.asarray(trainingDf.loc[:, test.columns != 'image'])
print(y_train_final.shape)


#Get x_Data ready for model
x_data_final = np.asarray(x_data_train).astype('float32') / 255.
print(x_data_final.shape) #x_train





(131, 7)
(131, 450, 600, 3)


In [ ]:
wrap_model = keras.wrappers.scikit_learn.KerasClassifier(buildModel)

params = {
    "epochs": [5, 10],
    "batch_size": [5, 10, 100]   
}

rnd_search_cv = RandomizedSearchCV(wrap_model, params, n_iter=5, cv=3)
rnd_search_cv.fit(X = x_data_final, y = y_train_final)

In [ ]:
#epochs = 15
history = m.fit(x_data_final, y_train_final, epochs=15, batch_size=64)

#look for keras tunner

Epoch 1/15
3/3 [==============================] - 117s 29s/step - loss: 20.0184 - accuracy: 0.3817
Epoch 2/15
3/3 [==============================] - 105s 27s/step - loss: 366.4836 - accuracy: 0.2977
Epoch 3/15
3/3 [==============================] - 98s 26s/step - loss: 2.2101 - accuracy: 0.4198
Epoch 4/15
3/3 [==============================] - 94s 23s/step - loss: 1.9190 - accuracy: 0.6718
Epoch 5/15
3/3 [==============================] - 101s 26s/step - loss: 1.9099 - accuracy: 0.6718
Epoch 6/15
3/3 [==============================] - 99s 29s/step - loss: 1.8964 - accuracy: 0.6718
Epoch 7/15
3/3 [==============================] - 96s 27s/step - loss: 1.8849 - accuracy: 0.6718
Epoch 8/15
3/3 [==============================] - 106s 27s/step - loss: 1.8734 - accuracy: 0.6718
Epoch 9/15
3/3 [==============================] - 107s 28s/step - loss: 1.8632 - accuracy: 0.6718
Epoch 10/15
3/3 [==============================] - 110s 32s/step - loss: 1.8518 - accuracy: 0.6718
Epoch 11/15
1/3 [===

### Autoencoder


In [ ]:
#Process train data
x_data_final_train = np.asarray(x_data_train)
x_train = x_data_final_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))

In [ ]:
#Process test data
x_data_final_test = np.asarray(x_data_test)
x_test = x_data_final_test.astype('float32') / 255.
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [ ]:
x_test.shape

In [ ]:
encoding_dim = 64  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(810000,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(810000, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

In [ ]:
#This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))

# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]

# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))


In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                verbose = False,
                validation_data=(x_test, x_test))

In [ ]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


In [ ]:
# Use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 9  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(450, 600,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(450, 600,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Autoencoder for denoising.

In [ ]:
x_train_new = x_data_final_train.astype('float32') / 255.
x_test_new = x_data_final_test.astype('float32') / 255.
x_train_new = np.reshape(x_train_new, (len(x_train), 450, 600, 3))
x_test_new = np.reshape(x_test_new, (len(x_test), 450, 600, 3))

noise_factor = 0.5
x_train_noisy = x_train_new + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train_new.shape) 
x_test_noisy = x_test_new + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test_new.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
x_train_noisy.shape

In [ ]:
x_train_new.shape

In [ ]:
n = 8
plt.figure(figsize=(20, 2))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(x_test_noisy[i].reshape(450, 600, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
from tensorflow.keras import layers

input_img = keras.Input(shape=(450, 600, 3))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train_noisy, x_train_new,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test_new))

In [ ]:
decoded_imgs = autoencoder.predict(x_test_noisy)

n = 8
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test_noisy[i].reshape(450, 600, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(450, 600, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()